### Standard preamble
Please run the following three cells to get the environment set up 
and the necessary packages installed.
After running the first cell, you will need to "Restart Runtime"
as per the message prompt that will appear.
After that, proceed to run the second and third code cells below.

In [ ]:
# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.9 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    ! git clone https://github.com/monash-emu/WPRO_wip.git --branch main
    %cd WPRO_wip
    %pip install -e ./
except:
    pass

In [ ]:
import pandas as pd
from datetime import datetime
from constants import PROJECT_PATH
from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput
pd.options.plotting.backend = 'plotly'

## Epidemiological explanation
This is incomplete

In [ ]:
iso = 'MYS'

In [ ]:
deaths_data = pd.read_csv(PROJECT_PATH / 'data/new_deaths.csv', index_col=0)[iso]
deaths_data.index = pd.to_datetime(deaths_data.index)
approx_pops = {
    'MYS': 33e6,
    'PHL': 114e6,
    'VNM': 97e6,
}

In [ ]:
analysis_start_date = datetime(2021, 8, 1) 
analysis_end_date = datetime(2022, 6, 1)
epi_model = CompartmentalModel(
    [analysis_start_date, analysis_end_date],
    ['susceptible', 'exposed', 'infectious', 'recovered'],
    ['infectious'],
    ref_date=datetime(2019, 12, 31),
)
epi_model.add_infection_frequency_flow('infection', Parameter('contact_rate'), 'susceptible', 'exposed')
epi_model.add_transition_flow('progression', 1.0 / Parameter('incubation_period'), 'exposed', 'infectious')
epi_model.add_transition_flow('recovery', 1.0 / Parameter('infectious_period'), 'infectious', 'recovered')
epi_model.set_initial_population({'susceptible': approx_pops[iso], 'infectious': 1.0})
epi_model.request_output_for_flow('recovery', 'recovery', save_results=False)
epi_model.request_function_output('deaths', DerivedOutput('recovery') * Parameter('ifr'));

In [ ]:
parameters = {
    'contact_rate': 0.4,
    'incubation_period': 5.0,
    'infectious_period': 5.0,
    'ifr': 0.0002,
}
epi_model.run(parameters)

In [ ]:
plot_start_date = datetime(2021, 10, 1)
comparison_df = pd.concat([epi_model.get_derived_outputs_df(), deaths_data])
comparison_df.columns = ['modelled', 'reported']
comparison_plot = comparison_df.plot()
comparison_plot.update_xaxes(range=(plot_start_date, analysis_end_date))